In [ ]:
import pandas as pd
import requests as r
import numpy as np
import json
import plotly.graph_objects as go

In [ ]:
# Getting the dataframe with APL data
apl=pd.read_csv('../data/medical_desert_clean.csv') 

geo_apl = apl[['CODGEO','Communes','APL']]
geo_apl = geo_apl.rename(columns={'CODGEO':'insee_com'})
geo_apl.head()

In [ ]:
# Getting geojson for every cities in France
url_geo="https://data.opendatasoft.com/api/v2/opendatasoft/datasets/correspondance-code-insee-code-postal%40public/exports/geojson?rows=-1&timezone=UTC&pretty=false"

geojson=r.get(url_geo).json() 
geojson

In [ ]:
# Getting a reduced dataframe to iterate more rapidly on the map (because it takes time to run)

# getting a geojson with onyl 10 rows
sub_geo="https://data.opendatasoft.com/api/v2/opendatasoft/datasets/correspondance-code-insee-code-postal%40public/exports/geojson?rows=10&timezone=UTC&pretty=false"

sub_geojson=r.get(sub_geo).json() 
subjson = pd.json_normalize(sub_geojson['features'])

# gettng the insee_com values from the request and filter the geo_apl dataframe
val = subjson['properties.insee_com'].values
sub_geo_apl = geo_apl[geo_apl.insee_com.isin(val)]


In [ ]:
# Building the interactive map using Plotly
fig = go.Figure(go.Choroplethmapbox(geojson=geojson, locations=geo_apl.insee_com, z=geo_apl.APL,
                                    text=geo_apl.Communes,
                                    featureidkey="properties.insee_com",
                                    colorscale="Viridis", zmin=0, zmax=4.5,
                                    marker_opacity=0.9, marker_line_width=0,
                                    hoverinfo='text+z'))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=5, mapbox_center = {"lat": 48.862725, "lon": 2.287592})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("interactive_map.html")
fig.show()